## Renombrado Automático

#### Generar modelo

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
import numpy as np

# Ruta del archivo CSV
ruta_archivo = 'dataset_clean.csv'

# Cargar el CSV como un DataFrame
df = pd.read_csv(ruta_archivo)

X = np.array([[int(x) for x in list(fila)[:-1]] for _,fila in df.iterrows()])
y = np.array([str(i) for i in list(df["Categoria"])])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train, y_train)
print('end')

print(f"Mejores hiperparámetros: {grid_search.best_params_}")

best_model = grid_search.best_estimator_

y_pred_dt = best_model.predict(X_test)
# Métricas para árbol de decisión
print("\nResultados para Árbol de Decisión:")
print("Accuracy:", accuracy_score(y_test, y_pred_dt))

C:\Users\Emanuel H\AppData\Local\Temp\ipykernel_13460\1584397684.py:12: DtypeWarning: Columns (780) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_archivo)


end
Mejores hiperparámetros: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 50}

Resultados para Árbol de Decisión:
Accuracy: 0.9987778796211427


In [ ]:
# Espera de carga de la página (Tiempo de espera de carga de la página y la búsqueda)
tiempo_espera = 0.5 # segundos

# Número de capchas que quiere descargar
cant_catchas = 2

In [3]:
def listasDeUnaImagen(filename: str) -> np.ndarray:
    return np.array([[0]])

In [ ]:
# código para descargar las imágenes capcha

# !pip install selenium # para instalar selenium

from selenium import webdriver
from selenium.webdriver.common.by import By
from pathlib import Path
from IPython.display import clear_output
import cv2
import matplotlib.pyplot as plt
import os
import time

# Almacenamiento de las catchas renombradas 
out_dir_rename = "catchas_renombrados"
os.makedirs(out_dir_rename, exist_ok=True)

# Configuración de Selenium
options = webdriver.FirefoxOptions()
options.add_argument("--headless")  # Ejecuta Chrome sin GUI
driver = webdriver.Firefox(options=options)
driver_img = webdriver.Firefox(options=options)

# Almacenamiento temporal de imágenes sin nombre
out_dir_tem = "descarga"
os.makedirs(out_dir_tem, exist_ok=True)

# Ingresar a la página del capcha
captcha_url = "https://www2.sunarp.gob.pe/consulta-vehicular/inicio"
driver.get(captcha_url) # Abrir página
time.sleep(tiempo_espera)

for i in range(cant_catchas):
    clear_output(wait=True)

    # Navegar a la URL del CAPTCHA
    image_element = driver.find_element(By.ID, "image")
    image_url = image_element.get_attribute("src")
    driver_img.get(image_url)

    # Guardar la imagen en el disco
    filename = out_dir_tem + "/" + ('00000' + str(i+1))[-5:] + '.png'
    cap = driver_img.find_element(by=By.XPATH, value="/html/body/img").screenshot(filename)
    if cap:
        print(filename, "descargado correctamente")
    else:
        continue
    
    carpeta = Path(out_dir_tem)
    archivo = [arc for arc in carpeta.iterdir()][0]
    if archivo.is_file():
        print("Archivo encontrado:", archivo)
        image = cv2.imread(archivo)
        
        ## Mostrar imagen capcha
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()
        
        name_catcha = "".join(best_model.predict(listasDeUnaImagen))

        # Enviar el nombre del catcha a la página para probar que es correcto
        plata_input = driver.find_element(By.ID, "nroPlaca")
        plata_input.clear()
        plata_input.send_keys("SSSSSS")
        
        capcha_input = driver.find_element(By.ID, "codigoCaptcha")
        capcha_input.clear()
        capcha_input.send_keys(name_catcha)

        submit_button = driver.find_element(By.XPATH, "//button/span[text()=' Realizar Busqueda ']")
        submit_button.click()
        time.sleep(tiempo_espera)

        text_output = driver.find_element(By.ID, "swal2-title")
        
        if text_output.text == "Captcha inválido":
            print("capcha ingresado:", name_catcha)
            print("El nombre del capcha es incorrecto")
            boton_ok = driver.find_element(By.XPATH, "//button[@type='button' and text()='OK']")
            boton_ok.click()
            time.sleep(3)
        else:
            boton_aceptar = driver.find_element(By.XPATH, "//button/span[text()='Aceptar']")
            boton_aceptar.click()

            output_path = out_dir_rename + "/" + name_catcha + ".png"
            cv2.imwrite(output_path, image)

        ruta_archivo = archivo

        # Verificar si el archivo existe antes de eliminarlo
        if os.path.exists(archivo):
            os.remove(archivo)
        else:
            print(f"El archivo no existe: {archivo}")

# Cerrar el navegador
driver.quit()
driver_img.quit()